# IBM Capstone Project

## Introduction

Most of us in the business world have heard the phrase at some point ‘location, location, location’. Considering the potential location of a business is one of the more important parts of starting one. After all, almost any firm based in Times Square is going to do better overall than if that same firm were to be placed in a corn field in South Dakota. Although no one questions the importance of location, there are other factors that go into whether a business succeeds or fails. If a firm were to neglect say, customer service in the name of a high value location, then cultivating a core clientele would be a challenge regardless of how many people pass by your front door. Would it not be handy if there was a way to quantify exactly how important location is? That’s what this capstone aims to find out. This project is aimed to interest any business owner that is either opening a new location, or even just starting out. 

## The Data

Since it is not possible for an individual such as myself to know the intimate financial details of various businesses in order to deem said business a success or failure, a creative way to determine success of a business must be found. It just so happens that Foursquare’s API has a attribute in their json that returns a rating of the venue. Using this field, it is possible to determine whether or not a business has a strong core clientele or not, so this is the metric that will be used for the success of a business. Because of its density and diversity of venues, New York City and some of its surrounding areas will be the subject of the explore query. All of the venues that result from the initial query will require an additional query to get their respective ratings, since that information is not available in the explore query’s results. Then all this information will be placed into a SQL database. Although the data will essentially be pre-cleaned using this method the downsides are the query limits in Foursquare’s API, so the data will have to be gathered over the course of a multiple days as the limits refresh (thus the reason a SQL database is required as opposed to importing the data directly into a pandas dataframe).  

##### Here is the program used to query Foursquare and load the data into a database. It is unnecessary to run here since the data has already been gathered (it won't work anyway since the client id and secret have been changed for obvious reasons).

In [ ]:
#Builds a sqlite file to store a venues latlon and rating

import sqlite3 as sq
import json as js
import requests
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180323'

db = sq.connect('venue_tips.sql')
cr = db.cursor()

#user input for location selection
first_time = input('Is this your first time running this program? y/n: ')
target = input('Enter the area you wish to explore (format example: "40.7243,-74.0018"), or leave blank: ')


#queries for looking up venue information
def explore_query(loc):
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = dict(
    client_id = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    v = '20180323',
    ll = loc,
    radius = 1500, #radius is in meters
    limit = 600
    )
    resp = requests.get(url = url, params = params)
    data = js.loads(resp.text)
    return data

def venue_query(ven):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(ven, CLIENT_ID, CLIENT_SECRET, VERSION)
    resp = requests.get(url = url)
    data = js.loads(resp.text)
    return data



#create table for latlon, rating
if first_time == 'y' or first_time == 'Y':
    cr.execute('''CREATE TABLE venues (id text unique, latitude real, longitude real, rating real)''')
    db.commit()

#Gets venue location data to load into db
if target != '':
    print('Querying Foursquare....', end = '')
    jaysahn = explore_query(target)
    if jaysahn['meta']['code'] != 200:
        print('max calls exceeded, come back tomorrow')
        quit()
    print('done')
    #loops through venues in results, loading into db
    count = 0
    for ven in jaysahn['response']['groups'][0]['items']:
        cr.execute('''INSERT OR IGNORE INTO venues VALUES(?, ?, ?, 1234.0)''',
            (ven['venue']['id'], ven['venue']['location']['lat'], ven['venue']['location']['lng']))
        count += 1
        if count % 10 == 0:
            db.commit()
            print('loading database')
    db.commit()


#for loop that pulls venue id, sees if there is a rating, if not, sends the venue query and loads the info
#upon completion, says something like 'done'
cr.execute('''SELECT id FROM venues WHERE rating == 1234.0''')
vens = cr.fetchall()
if vens == []:
    print('All Done!')
    db.commit()
    db.close()
    quit()

#gets ratings and updates db
count = 0
for idd in vens:
    new = venue_query(idd[0])
    if new['meta']['code'] != 200:
        print('max calls exceeded, come back tomorrow')
        db.commit()
        db.close()
        quit()
    try:
        new_rating = new['response']['venue']['rating']
    except KeyError:
        cr.execute('''DELETE FROM venues WHERE id = ?''', (idd[0],))
        continue
    cr.execute('UPDATE venues SET rating = ? WHERE id = ?', (new_rating, idd[0]))
    count += 1
    if count % 10 == 0:
        db.commit()
        print('updating database')






db.commit()
db.close()


In [47]:
import pandas as pd
import sqlite3 as sql

db = sql.connect('venue_tips.sql')    
df = pd.read_sql('''SELECT * FROM venues''', db)


In [48]:
df.describe()

,latitude,longitude,rating
count,1040.000000,1040.000000,1040.000000
mean,40.733064,-73.955528,8.025481
std,0.048766,0.148525,1.015375
min,40.584406,-74.346905,5.400000
25%,40.714053,-74.037088,7.300000
50%,40.727742,-73.993157,8.000000
75%,40.755666,-73.931468,9.000000
max,41.110787,-73.311905,9.700000


## Methodology

Out of a little over 1000 venues gathered from New York and some surrounding areas, the average rating is almost exactly 8. Based on some of the other information provided in the describe output of the dataframe, it is possible designate a venue 'average', 'below average', and 'above average' based on where most of the ratings end up being. A look at the 25% mark of the data reveals that the first quarter of the venues are at or below 7.3. This line will be used to delineate the 'below average' label. At the 75% mark, the last quarter of the data has a rating of 9 or above. This rating will dlineate the 'above average' label. On the map shown below, 'above average' venues will have a blue marker, 'average' venues will have a green marker, and the 'below average' venues will have a red marker.

In [49]:
import folium as f
import pandas as pd
import sqlite3 as sql
m = f.Map(location=[40.7128, -74.0060], zoomstart = 12)

def color_define(rating):
    if rating < 7.3: return 'red'
    elif rating < 9: return 'green'
    else: return 'blue'
    
for lat, lon, rat in zip(df['latitude'], df['longitude'], df['rating']):
    f.Marker([lat, lon], icon = f.Icon(color = color_define(rat), icon = 'icon')).add_to(m)
m

To the surprise of no one, the map indicates a correlation between location and rating. This however was not the objective of this exercise. The challenge is to find a way to quantify the strength of location's effect on a business's success (which we are using rating to measure). Looking at the map through exclusively a geographical lens, it is possible to see roughly three clusters. However, the large mass in the center should be split two ways since it is so large in comparison to the other two clusters. When the venues are segmented using a k-means algorithm based of their coordinates alone, the resulting map looks like this: 

In [50]:
from sklearn.cluster import KMeans
from sklearn import preprocessing

#makes dataframe
geo = df[['latitude', 'longitude']]

#makes model
geo_segment = KMeans(n_clusters = 4, random_state=0).fit(geo)
geo.loc[:,'labels'] = geo_segment.labels_

#makes the map, assigns colors
m_geo = f.Map(location=[40.7128, -74.0060], zoomstart = 12)

def geocolor_define(label):
    if label == 0: return 'red'
    elif label == 1: return 'green'
    elif label == 2: return 'purple'
    else: return 'blue'

for lat, lon, lab in zip(geo['latitude'], geo['longitude'], geo['labels']):
    f.Marker([lat, lon], icon = f.Icon(color = geocolor_define(lab), icon = 'icon')).add_to(m_geo)
m_geo



c:\users\kevin\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\kevin\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Now, in an imaginary world where location is not at all a factor in determining the success of a business, the distribution of ratings would be even in a given cluster. There would be a similar numbers of highly rated businesses and poorly rated businesses. Recalling back to when the venues were labeled 'average', 'below average', and 'above average', it would be reasonable to expect in this imaginary world that 50% of the venues would be average, and the other two categories would each have a 25% presence in given cluster. The strength of the effect of location on a venue's rating can be measured by comparing a cluster's true collective ratings against this imaginary world. An example with nicer numbers would make understanding what will done to measure location's strength a ton easier. Say there are two clusters with 100 venues each. Cluster One has 50 venues that are  are 'above average', 25 are 'below average', and 25 are 'average'. Cluster Two has 70 venues that are 'below average', 25 venues that are 'average', and 5 that are 'above average'. Since it has been established that location does pull a venue's rating either up or down, it can be assumed that a particular location has an inherent 'above average', 'average', or 'below average' quality to it. This quality can be ascertained by looking at which quality has the largest gap between its actual value and expected value. In Cluster One's case, it would have a 'above average' location, and Cluster Two would have a 'below average' location. The difference between Cluster One's amount of 'above average' venues and its expected value from the imaginary world (25) is 25. This means that the location effectively doubled the amount of venues considered 'above average'. If recorded as a multiplier to the expected number of 'above average' venues, this would be 100% (from now on this number will be referred to as Location Pull). Performing the same operation on Cluster Two yields 180%, indicating a very strong pull. To get a final representive number of the strength of location in general, the two Location Pulls are averaged together to get 140%. Now that the process is hopefully understood, it is time to apply it to the real data.


## Results

To get the final Location Pull values, all that needs to be done is to define a function to classify the quality of clusters, and then calculate the Location Pull based on that label. Here is what the function looks like:

In [51]:
#adds geographic labels to initial data
df.loc[:, 'labels'] = geo_segment.labels_



#takes in cluster label from geographic segmentation, outputs a Location Pull
def cluster_pull(labell):
    tempdf = df.loc[df['labels'] == labell, :]
    
    abv = 0
    avg = 0
    blw = 0
    total = 0
    for rat, val in tempdf['rating'].value_counts().items():
        if rat < 7.3: blw += val 
        elif rat < 9: avg += val
        else: abv += val
        total += val
    abvgap = abv - (total * .25)
    avggap = avg - (total * .5)
    blwgap = blw - (total * .25)
    gaps = [abvgap, avggap, blwgap]
    biggest = max(gaps)
    ind = gaps.index(biggest)
    if ind == 0: return (abvgap / total) * 100   #'above average'
    elif ind == 1: return (avggap / total) * 100  #'average'
    else: return (blwgap / total) * 100 #'below average'



Now in order to get the final value for the strength of location in general, all the values must be averaged together. Here is the final result: 

In [52]:
import statistics

avglist = []
for i in range(4):
    avglist.append(cluster_pull(i))
    
print(str(round(statistics.mean(avglist), 2)) + '% Location Pull')

15.87% Location Pull


## Discussion

Now before getting too excited about this number, there is one limitation to this project that must be discussed, and that is the data itself. Casting a widespread yet diverse net proved difficult during the course of this project, due to the fact that the Foursquare API will present the best venues first, which makes sense from their perspective, but resulted in inaccurate data. In order to get a more accurate representation, more specific queries had to be made to the API so that there was no choice but to present ALL venues, no matter how poorly they were rated. This process consumed the query limit rather quickly, making data gathering more tedious than it truly needed to be. However, in a real business situation, it is more likely that a premium account would be made available for use, lifting those query limits and freeing up precious time to be more systematic and therefore accurate. Does all of this mean that the work done here is useless? Not exactly.

## Conclusion

The mission of this project was to get a definitive number that can value the strength of location on a business's success. A trustable definitive number was not reached. HOWEVER, like many instances throughout history, something else even better may have been unearthed by accident, and is good reason to not relegate this project to the recycle bin. While the result is untrustworthy, process still is. If approached by a client with a specific location that he/she is uncertain about, the capability to tell them both the quality of the location and the magnitude of its effect on rating is absolutely there once all of the surrounding venues in the location are included in a database of its own. This is far more useful than the initial goal because the process used here is immediately applicable to the real world. In summary, this project looked for copper and potentially found gold.